# From area, try calculate the sides of it via circles and ellipse

In [106]:
import numpy as np

In [107]:
def get_circle_radius_from_area(area):
    """
    Calculates the radius of a circle given its area.

    Parameters:
    area (float): The area of the circle.

    Returns:
    float: The radius of the circle.
    """
    radius = np.sqrt(area / np.pi)
    return radius

def resize_ellipse_radius_with_fixed_area(original_area, fixed_radius):
    """
    Resizes an ellipse by changing one radius while preserving the other radius and maintaining the same area.

    Parameters:
    original_area (float): The original area of the ellipse.
    fixed_radius (float): The radius to be preserved (can be either the minor or major radius).

    Returns:
    float: The new radius of the ellipse that will maintain the original area.

    Raises:
    ValueError: If the original area or fixed radius is not positive.
    """
    if original_area <= 0:
        raise ValueError("The original area must be a positive number.")
    if fixed_radius <= 0:
        raise ValueError("The fixed radius must be a positive number.")

    # Calculate the new radius needed to maintain the same area
    new_radius = original_area / (np.pi * fixed_radius)

    return new_radius


In [108]:
# Example usage
a = 28841
b = 34617
original_area = a*b
fixed_radius = get_circle_radius_from_area(original_area)

# sanity check that same area and same radius is same result
new_radius = resize_ellipse_radius_with_fixed_area(original_area, fixed_radius)

print(f"Original Area: {original_area}")
print(f"Minor Radius: {fixed_radius}")
print(f"Major Radius: {new_radius}")

# Start the search for better approximity
# 1. first, changed fixed_radius with a radius_rate and maintaining the same area.
radius_rate = np.pi/4
fixed_minor_radius = fixed_radius * radius_rate
new_major_radius = resize_ellipse_radius_with_fixed_area(original_area, fixed_minor_radius)

print('\nResult after radius is changed and area maintained')
print(f"Area: {original_area}")
print(f"Minor Radius: {fixed_minor_radius}")
print(f"Major Radius: {new_major_radius}")

# 2. change the area with an area_rate and than calculate a changed radius with this area
area_rate = np.pi/4
new_area = original_area * area_rate
fixed_major_radius = resize_ellipse_radius_with_fixed_area(new_area, fixed_minor_radius)

print('\nResult after area is changed and minor radius is maintained')
print(f"Area: {new_area}")
print(f"Minor Radius: {fixed_minor_radius}")
print(f"Major Radius: {fixed_major_radius}")

# 3. Create a bounding rectangle with the fixed minor radius and calculate its area
minor_square_side = 2 * fixed_minor_radius
major_square_side = 2 * fixed_major_radius
bounding_area = minor_square_side * major_square_side

# 4. Convert to int
fixed_minor_radius_int = int(fixed_minor_radius)
fixed_major_radius_int = int(fixed_major_radius)
area_int = 4* fixed_minor_radius_int * fixed_major_radius_int

print('\nCalculate bounding rectangle area')
print(f"area_val: {original_area}")
print(f"area_pre: {bounding_area}")
print(f"area_int: {area_int}")
print(f"x_val: {a}")
print(f"x_pre: {minor_square_side}")
print(f"x_int: {2*fixed_minor_radius_int}")
print(f"y_val: {b}")
print(f"y_pre: {major_square_side}")
print(f"y_int: {2*fixed_major_radius_int}")


Original Area: 998388897
Minor Radius: 17826.86332957176
Major Radius: 17826.86332957176

Result after radius is changed and area maintained
Area: 998388897
Minor Radius: 14001.18571818298
Major Radius: 22697.86734979991

Result after area is changed and minor radius is maintained
Area: 784132806.0602041
Minor Radius: 14001.18571818298
Major Radius: 17826.86332957176

Calculate bounding rectangle area
area_val: 998388897
area_pre: 998388897.0
area_int: 998327304
x_val: 28841
x_pre: 28002.37143636596
x_int: 28002
y_val: 34617
y_pre: 35653.72665914352
y_int: 35652


In [109]:
# Calculate floor and ceil for both radii
fixed_minor_radius_floor = np.floor(fixed_minor_radius)
fixed_major_radius_floor = np.floor(fixed_major_radius)

fixed_minor_radius_ceil = np.ceil(fixed_minor_radius)
fixed_major_radius_ceil = np.ceil(fixed_major_radius)

# Calculate areas for all combinations
area_floor_floor = 4 * fixed_minor_radius_floor * fixed_major_radius_floor
area_floor_ceil = 4 * fixed_minor_radius_floor * fixed_major_radius_ceil
area_ceil_floor = 4 * fixed_minor_radius_ceil * fixed_major_radius_floor
area_ceil_ceil = 4 * fixed_minor_radius_ceil * fixed_major_radius_ceil

# Print or return the results
print(f"Area (floor * floor): {area_floor_floor}")
print(f"Area (floor * ceil) : {area_floor_ceil}")
print(f"Area (ceil * floor) : {area_ceil_floor}")
print(f"Area (ceil * ceil)  : {area_ceil_ceil}")

# Calculate fault rates (as percentages)
fault_rate_floor_floor = np.abs(area_floor_floor - original_area) / original_area * 100
fault_rate_floor_ceil = np.abs(area_floor_ceil - original_area) / original_area * 100
fault_rate_ceil_floor = np.abs(area_ceil_floor - original_area) / original_area * 100
fault_rate_ceil_ceil = np.abs(area_ceil_ceil - original_area) / original_area * 100

# Create a dictionary to store the fault rates with their corresponding areas
fault_rates = {
    'floor * floor': fault_rate_floor_floor,
    'floor * ceil': fault_rate_floor_ceil,
    'ceil * floor': fault_rate_ceil_floor,
    'ceil * ceil': fault_rate_ceil_ceil
}

# Find the combination with the minimum fault rate
best_match = min(fault_rates, key=fault_rates.get)

# Print the fault rates
print("\nFault rates for each combination:")
print(f"floor * floor: {fault_rate_floor_floor:.4f}%")
print(f"floor * ceil : {fault_rate_floor_ceil:.4f}%")
print(f"ceil * floor : {fault_rate_ceil_floor:.4f}%")
print(f"ceil * ceil  : {fault_rate_ceil_ceil:.4f}%")

# Print the best match
print(f"\nBest match: {best_match} with a fault rate of {fault_rates[best_match]:.6f}%")

# Ellipse area of the bounded rectangle can approximately be calculated as
ellipse_area = original_area * np.pi / 4  # Approximating based on original area and pi/4 factor

# Ellipse area calculation with the fixed radii from the best match gives the predicted area
if best_match == 'floor * floor':
    ellipse_pred = np.pi * fixed_minor_radius_floor * fixed_major_radius_floor
elif best_match == 'floor * ceil':
    ellipse_pred = np.pi * fixed_minor_radius_floor * fixed_major_radius_ceil
elif best_match == 'ceil * floor':
    ellipse_pred = np.pi * fixed_minor_radius_ceil * fixed_major_radius_floor
else:  # best_match == 'ceil * ceil'
    ellipse_pred = np.pi * fixed_minor_radius_ceil * fixed_major_radius_ceil

# Ratio of ellipse area to approximate area
ellipse_area_ratio = np.abs(ellipse_pred - ellipse_area) / ellipse_area * 100

# Print the calculated ellipse area, predicted ellipse area, and their ratio
print(f"\nEllipse area         : {ellipse_area:.2f}")
print(f"Predicted ellipse area: {ellipse_pred:.2f}")
print(f"Ellipse area ratio   : {ellipse_area_ratio:.6f}%")

Area (floor * floor): 998327304.0
Area (floor * ceil) : 998383308.0
Area (ceil * floor) : 998398608.0
Area (ceil * ceil)  : 998454616.0

Fault rates for each combination:
floor * floor: 0.0062%
floor * ceil : 0.0006%
ceil * floor : 0.0010%
ceil * ceil  : 0.0066%

Best match: floor * ceil with a fault rate of 0.000560%

Ellipse area         : 784132806.06
Predicted ellipse area: 784128416.47
Ellipse area ratio   : 0.000560%


## Calculate new major radius when minor radius is changed with the radius rate
Convert it to functions

In [110]:
# Example usage
a = 28841
b = 34617
original_area = a*b
fixed_radius = get_circle_radius_from_area(original_area)

# sanity check that same area and same radius is same result
new_radius = resize_ellipse_radius_with_fixed_area(original_area, fixed_radius)

print(f"Original Area: {original_area}")
print(f"Minor Radius: {fixed_radius}")
print(f"Major Radius: {new_radius}")

Original Area: 998388897
Minor Radius: 17826.86332957176
Major Radius: 17826.86332957176


In [111]:
def calculate_ellipse_properties(a, b, minor_radius, minor_radius_rate=np.pi/4):
    original_area = a * b

    # Calculate fixed minor radius with the radius_rate
    fixed_minor_radius = minor_radius * minor_radius_rate

    # Approximate area of ellipse as the bounded rectangle
    approx_ellipse_area = original_area * np.pi/4
    # Calculate a new radii from this area when the other radius is known
    fixed_major_radius = resize_ellipse_radius_with_fixed_area(approx_ellipse_area, fixed_minor_radius)

    # Create a bounding rectangle from the fixed radii and calculate its area
    minor_square_side = 2 * fixed_minor_radius
    major_square_side = 2 * fixed_major_radius
    bounding_area = minor_square_side * major_square_side

    # Calculate fault rates (as percentages)
    area_fault_rate = np.abs(bounding_area - original_area) / original_area * 100

    return {
        "a": a,
        "b": b,
        "original_area": original_area,
        "bounding_area": bounding_area,
        "fault_rate": area_fault_rate,
        "minor_radius": fixed_minor_radius,
        "major_radius": fixed_major_radius,
        "x_pre": minor_square_side,
        "y_pre": major_square_side,
    }

def print_calculated_properties(props):
    # Extract the necessary values from the props dictionary
    minor_radius = props["minor_radius"]
    major_radius = props["major_radius"]
    original_area = props["original_area"]

    # Calculate floor and ceil for both radii
    fixed_minor_radius_floor = np.floor(minor_radius)
    fixed_major_radius_floor = np.floor(major_radius)
    fixed_minor_radius_ceil = np.ceil(minor_radius)
    fixed_major_radius_ceil = np.ceil(major_radius)

    # Calculate areas for all combinations
    area_floor_floor = 4 * fixed_minor_radius_floor * fixed_major_radius_floor
    area_floor_ceil = 4 * fixed_minor_radius_floor * fixed_major_radius_ceil
    area_ceil_floor = 4 * fixed_minor_radius_ceil * fixed_major_radius_floor
    area_ceil_ceil = 4 * fixed_minor_radius_ceil * fixed_major_radius_ceil

    # Print Area
    print(f"Area (original)     : {original_area}")
    print(f"Area (floor * floor): {area_floor_floor}")
    print(f"Area (floor * ceil) : {area_floor_ceil}")
    print(f"Area (ceil * floor) : {area_ceil_floor}")
    print(f"Area (ceil * ceil)  : {area_ceil_ceil}")

    # Calculate fault rates (as percentages)
    fault_rate_floor_floor = np.abs(area_floor_floor - original_area) / original_area * 100
    fault_rate_floor_ceil = np.abs(area_floor_ceil - original_area) / original_area * 100
    fault_rate_ceil_floor = np.abs(area_ceil_floor - original_area) / original_area * 100
    fault_rate_ceil_ceil = np.abs(area_ceil_ceil - original_area) / original_area * 100

    # Create a dictionary to store the fault rates with their corresponding areas
    fault_rates = {
        'floor * floor': fault_rate_floor_floor,
        'floor * ceil': fault_rate_floor_ceil,
        'ceil * floor': fault_rate_ceil_floor,
        'ceil * ceil': fault_rate_ceil_ceil
    }

    # Find the combination with the minimum fault rate
    best_match = min(fault_rates, key=fault_rates.get)

    # Print the fault rates
    print("\nFault rates for each combination:")
    print(f"floor * floor: {fault_rate_floor_floor:.4f}%")
    print(f"floor * ceil : {fault_rate_floor_ceil:.4f}%")
    print(f"ceil * floor : {fault_rate_ceil_floor:.4f}%")
    print(f"ceil * ceil  : {fault_rate_ceil_ceil:.4f}%")

    # Print the best match
    print(f"\nBest match: {best_match} with a fault rate of {fault_rates[best_match]:.6f}%")

    # Set fixed_minor_radius and fixed_major_radius based on the best match
    if best_match == 'floor * floor':
        fixed_minor_radius = fixed_minor_radius_floor
        fixed_major_radius = fixed_major_radius_floor
    elif best_match == 'floor * ceil':
        fixed_minor_radius = fixed_minor_radius_floor
        fixed_major_radius = fixed_major_radius_ceil
    elif best_match == 'ceil * floor':
        fixed_minor_radius = fixed_minor_radius_ceil
        fixed_major_radius = fixed_major_radius_floor
    else:  # best_match == 'ceil * ceil'
        fixed_minor_radius = fixed_minor_radius_ceil
        fixed_major_radius = fixed_major_radius_ceil

    # Ellipse area of the bounded rectangle can approximately be calculated as
    ellipse_area = original_area * np.pi / 4  # Approximating based on original area and pi/4 factor

    # Ellipse area calculation with the fixed radii from the best match gives the predicted area
    ellipse_pred = np.pi * fixed_minor_radius * fixed_major_radius

    # Ratio of ellipse area to approximate area
    ellipse_area_ratio = np.abs(ellipse_pred - ellipse_area) / ellipse_area * 100

    # Print the calculated ellipse area, predicted ellipse area, and their ratio
    print(f"\nEllipse area         : {ellipse_area:.2f}")
    print(f"Predicted ellipse area: {ellipse_pred:.2f}")
    print(f"Ellipse area ratio   : {ellipse_area_ratio:.6f}%")

    # Create a bounding rectangle with the fixed minor radius and calculate its area
    minor_square_side = 2 * fixed_minor_radius
    major_square_side = 2 * fixed_major_radius
    bounding_area = minor_square_side * major_square_side

    # Calculate fault rates (as percentages)
    area_fault_rate = np.abs(bounding_area - original_area) / original_area * 100

    return {
        "a": a,
        "b": b,
        "original_area": original_area,
        "bounding_area": bounding_area,
        "fault_rate": area_fault_rate,
        "minor_radius": fixed_minor_radius,
        "major_radius": fixed_major_radius,
        "x_pre": minor_square_side,
        "y_pre": major_square_side,
    }


In [112]:
a = 28841
b = 34617
original_area = a*b
fixed_radius = get_circle_radius_from_area(original_area)
# Example usage:
prop = calculate_ellipse_properties(a=a, b=b, minor_radius=fixed_radius)
data = print_calculated_properties(prop)

Area (original)     : 998388897
Area (floor * floor): 998327304.0
Area (floor * ceil) : 998383308.0
Area (ceil * floor) : 998398608.0
Area (ceil * ceil)  : 998454616.0

Fault rates for each combination:
floor * floor: 0.0062%
floor * ceil : 0.0006%
ceil * floor : 0.0010%
ceil * ceil  : 0.0066%

Best match: floor * ceil with a fault rate of 0.000560%

Ellipse area         : 784132806.06
Predicted ellipse area: 784128416.47
Ellipse area ratio   : 0.000560%


In [113]:
prop

{'a': 28841,
 'b': 34617,
 'original_area': 998388897,
 'bounding_area': 998388897.0,
 'fault_rate': 0.0,
 'minor_radius': 14001.18571818298,
 'major_radius': 17826.86332957176,
 'x_pre': 28002.37143636596,
 'y_pre': 35653.72665914352}

In [114]:
data

{'a': 28841,
 'b': 34617,
 'original_area': 998388897,
 'bounding_area': 998383308.0,
 'fault_rate': 0.0005598018985181082,
 'minor_radius': 14001.0,
 'major_radius': 17827.0,
 'x_pre': 28002.0,
 'y_pre': 35654.0}

In [115]:
prop = calculate_ellipse_properties(a=a, b=b, minor_radius=27502/2)
data = print_calculated_properties(prop)
data

Area (original)     : 998388897
Area (floor * floor): 998352000.0
Area (floor * ceil) : 998395200.0
Area (ceil * floor) : 998444440.0
Area (ceil * ceil)  : 998487644.0

Fault rates for each combination:
floor * floor: 0.0037%
floor * ceil : 0.0006%
ceil * floor : 0.0056%
ceil * ceil  : 0.0099%

Best match: floor * ceil with a fault rate of 0.000631%

Ellipse area         : 784132806.06
Predicted ellipse area: 784137756.42
Ellipse area ratio   : 0.000631%


{'a': 28841,
 'b': 34617,
 'original_area': 998388897,
 'bounding_area': 998395200.0,
 'fault_rate': 0.0006313171169009905,
 'minor_radius': 10800.0,
 'major_radius': 23111.0,
 'x_pre': 21600.0,
 'y_pre': 46222.0}

In [116]:
prop = calculate_ellipse_properties(a=a, b=b, minor_radius=28502/2)
data = print_calculated_properties(prop)
data


Area (original)     : 998388897
Area (floor * floor): 998281632.0
Area (floor * ceil) : 998326400.0
Area (ceil * floor) : 998370828.0
Area (ceil * ceil)  : 998415600.0

Fault rates for each combination:
floor * floor: 0.0107%
floor * ceil : 0.0063%
ceil * floor : 0.0018%
ceil * ceil  : 0.0027%

Best match: ceil * floor with a fault rate of 0.001810%

Ellipse area         : 784132806.06
Predicted ellipse area: 784118614.70
Ellipse area ratio   : 0.001810%


{'a': 28841,
 'b': 34617,
 'original_area': 998388897,
 'bounding_area': 998370828.0,
 'fault_rate': 0.0018098157996642866,
 'minor_radius': 11193.0,
 'major_radius': 22299.0,
 'x_pre': 22386.0,
 'y_pre': 44598.0}

# Update function

In [117]:
def calculate_ellipse_properties(a, b, minor_radius):
    original_area = a * b

    # Ellipse area of the bounded rectangle can approximately be calculated as
    approx_ellipse_area = original_area * np.pi/4

    # Calculate a new radii from this area when the other radius is known
    major_radius = resize_ellipse_radius_with_fixed_area(approx_ellipse_area, minor_radius)

    # area is calculated only for int value
    # Create a bounding rectangle from radii and calculate its area
    minor_square_side = int(2 * minor_radius)
    major_square_side = int(2 * major_radius)
    bounding_area = minor_square_side * major_square_side

    # Calculate fault rates (as percentages)
    area_fault_rate = np.abs(bounding_area - original_area) / original_area * 100

    return {
        "a": a,
        "b": b,
        "original_area": original_area,
        "bounding_area": bounding_area,
        "fault_rate": area_fault_rate,
        "minor_radius": minor_radius,
        "major_radius": major_radius,
        "x_pre": minor_square_side,
        "y_pre": major_square_side,
    }


In [118]:
# Example
a = 28841
b = 34617

In [119]:
prop1 = calculate_ellipse_properties(a=a, b=b, minor_radius=28000/2)
fault1 = prop1["fault_rate"]
prop2 = calculate_ellipse_properties(a=a, b=b, minor_radius=28585/2)
fault2 = prop2["fault_rate"]
prop3 = calculate_ellipse_properties(a=a, b=b, minor_radius=27650/2)
fault3 = prop3["fault_rate"]

print("Fault rates:")
print(f"fault1: {fault1:.10f}%")
print(f"fault1: {fault2:.10f}%")
print(f"fault1: {fault3:.10f}%")

print(prop1)
print(prop2)
print(prop3)

Fault rates:
fault1: 0.0020930722%
fault1: 0.0000602971%
fault1: 0.0002701352%
{'a': 28841, 'b': 34617, 'original_area': 998388897, 'bounding_area': 998368000, 'fault_rate': 0.0020930721548278594, 'minor_radius': 14000.0, 'major_radius': 17828.373160714287, 'x_pre': 28000, 'y_pre': 35656}
{'a': 28841, 'b': 34617, 'original_area': 998388897, 'bounding_area': 998388295, 'fault_rate': 6.029714491105764e-05, 'minor_radius': 14292.5, 'major_radius': 17463.510529998253, 'x_pre': 28585, 'y_pre': 34927}
{'a': 28841, 'b': 34617, 'original_area': 998388897, 'bounding_area': 998386200, 'fault_rate': 0.0002701352156563496, 'minor_radius': 13825.0, 'major_radius': 18054.04877034358, 'x_pre': 27650, 'y_pre': 36108}


In [120]:
# Example with large prime
a = 17895697
b = 34567891
original_area = a*b
radius_init = get_circle_radius_from_area(original_area)

In [121]:
calculate_ellipse_properties(a=a, b=b, minor_radius=radius_init*0.57)

{'a': 17895697,
 'b': 34567891,
 'original_area': 618616503265027,
 'bounding_area': 618616492527254,
 'fault_rate': 1.735772153398199e-06,
 'minor_radius': 7998539.0645717075,
 'major_radius': 19335296.679524053,
 'x_pre': 15997078,
 'y_pre': 38670593}

In [128]:
calculate_ellipse_properties(a=a, b=b, minor_radius=radius_init*0.638)

{'a': 17895697,
 'b': 34567891,
 'original_area': 618616503265027,
 'bounding_area': 618616473639962,
 'fault_rate': 4.788922513971158e-06,
 'minor_radius': 8952750.742450438,
 'major_radius': 17274481.359449383,
 'x_pre': 17905501,
 'y_pre': 34548962}